In [21]:
import glob
import geopandas as gpd
from shapely.ops import unary_union
import pandas as pd

In [22]:
years = ["2019","2020","2021" "2022"]
months = ["05","06","07","08"]
year = years[0]
resolution =  "res14"

In [23]:
def get_nieuwkoopseplassen_data(year):
        return glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[0]))\
        +glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[1]))\
        +glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[2]))\
        +glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[3]))

In [24]:
def merge_hexagon_for_year(ayear):
        year_geojsons = glob.glob(("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+"*.geojson").replace("year", ayear).replace("month", months[0]))\
        +glob.glob(("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+"*.geojson").replace("year", ayear).replace("month", months[1]))\
        +glob.glob(("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+"*.geojson").replace("year", ayear).replace("month", months[2]))\
        +glob.glob(("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+"*.geojson").replace("year", ayear).replace("month", months[3]))

        print(len(year_geojsons))
        gdf = []
        for a_year_geojson in year_geojsons:
            agdf = gpd.read_file(a_year_geojson.replace("\\","/"))
            agdf['date'] = a_year_geojson.replace("\\","/").split("/")[-1].split("_")[0]+"-"+a_year_geojson.replace("\\","/").split("/")[-1].split("_")[1]
            gdf.append(agdf)

        gdf =pd.concat(gdf)

        source_gdf = gpd.read_file("C:/repos/satellite-images-nso/input_data/Nieuwkoopse Plassen & De Haeck.geojson")

        unary_union_geometry = unary_union(gdf['geometry'])


        print("Area covered with merge:")
        print(unary_union_geometry.intersection(source_gdf['geometry']).area/source_gdf['geometry'].area[0])
        print(gdf['date'].value_counts())
    
        return gdf

In [25]:
def prioritize_high_with_date(df):
    # Find the index of 'High' value, if exists, and return the corresponding 'Date'
    high_index = df[df['label'] == "Waterplants"]
    if len(high_index) > 0:
        return high_index.iloc[0]
    else:
        # If 'High' is not found, return the first row
        return df.iloc[0]

In [26]:
for year in years:
    gdf = merge_hexagon_for_year(year)
    export_gdf = gdf.groupby(["hexagon_id"]).apply(prioritize_high_with_date).drop(['hexagon_id'], axis=1).reset_index()
    export_gdf.to_file('E:/output/nieuwkoopse_plassen/'+str(year)+'_bloom_period_merge_ndwi_ndvi_hexagons_'+resolution+'.geojson')

4
Area covered with merge:


C:\Users\pzhadmin\AppData\Local\Temp\2\ipykernel_2920\1626335456.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(unary_union_geometry.intersection(source_gdf['geometry']).area/source_gdf['geometry'].area[0])


0    0.999987
dtype: float64
date
20190826-111247    4808125
20190629-113714    4628980
20190629-113712    3648170
20190523-111841    3596080
Name: count, dtype: int64


C:\Users\pzhadmin\AppData\Local\Temp\2\ipykernel_2920\3043851683.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  export_gdf = gdf.groupby(["hexagon_id"]).apply(prioritize_high_with_date).drop(['hexagon_id'], axis=1).reset_index()


3
Area covered with merge:
0    0.669603
dtype: float64


C:\Users\pzhadmin\AppData\Local\Temp\2\ipykernel_2920\1626335456.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(unary_union_geometry.intersection(source_gdf['geometry']).area/source_gdf['geometry'].area[0])


date
20200713-111443    2425073
20200812-111845    1594213
20200713-111445     869503
Name: count, dtype: int64


C:\Users\pzhadmin\AppData\Local\Temp\2\ipykernel_2920\3043851683.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  export_gdf = gdf.groupby(["hexagon_id"]).apply(prioritize_high_with_date).drop(['hexagon_id'], axis=1).reset_index()


0


ValueError: No objects to concatenate